In [1]:
# Check GPU availability and specs
!nvidia-smi

import torch

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU Count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
  print(f"GPU name: {torch.cuda.get_device_name(0)}")
  print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


/bin/bash: line 1: nvidia-smi: command not found
CUDA available: False
GPU Count: 0


In [2]:
# Mount Drive
from google.colab import drive
import os

#Mount Drive
drive.mount('/content/drive')

#Dir. to project directory
project_dir = '/content/drive/MyDrive/LoRA finetuning/mlops-sd-african-art'
os.makedirs(project_dir, exist_ok = True)
os.chdir(project_dir)

print(f"Working Directory: {os.getcwd()}")

Mounted at /content/drive
Working Directory: /content/drive/MyDrive/LoRA finetuning/mlops-sd-african-art


In [3]:
# Data Collection
!mkdir -p mlops-sd-african-art/{data/training_images,notebooks,pipelines,steps,models,app,configd,tests}
!touch steps/{data_loader,model_trainer,model_evaluator,model_deployer}.py
!touch pipelines/{training_pipeline,evaluation_pipeline,deployment_pipeline}.py
!cd mlops-sd-african-art

In [4]:
!ls pipelines

deployment_pipeline.py	evaluation_pipeline.py	training_pipeline.py


In [5]:
!pip install diffusers transformers accelerate peft bitsandbytes
!pip install zenml mlflow datasets pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114

In [6]:
#Authenticate Huggingface
from huggingface_hub import notebook_login
notebook_login()

In [7]:
## Dependencies
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.loaders import AttnProcsLayers
from diffusers.models.attention_processor import LoRAAttnProcessor
from transformers import CLIPTextModel
import os
from pathlib import Path

/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [8]:
class LoRATrainer:
    """Fine-tune Stable Diffusion XL with LoRA on Colab"""

    def __init__(self,
                 model_id="stabilityai/stable-diffusion-xl-base-1.0",
                 output_dir="./lora_weights",
                 concept_name="african_art"):
        self.model_id = model_id
        self.output_dir = output_dir
        self.concept_name = concept_name

    def setup_lora_layers(self, unet, rank=4):
        """Add LoRA layers to UNet attention processors"""
        lora_attn_procs = {}
        for name in unet.attn_processors.keys():
            cross_attention_dim = None if name.endswith("attn1.processor") else unet.config.cross_attention_dim
            if name.startswith("mid_block"):
                hidden_size = unet.config.block_out_channels[-1]
            elif name.startswith("up_blocks"):
                block_id = int(name[len("up_blocks.")])
                hidden_size = list(reversed(unet.config.block_out_channels))[block_id]
            elif name.startswith("down_blocks"):
                block_id = int(name[len("down_blocks.")])
                hidden_size = unet.config.block_out_channels[block_id]

            lora_attn_procs[name] = LoRAAttnProcessor(
                hidden_size=hidden_size,
                cross_attention_dim=cross_attention_dim,
                rank=rank
            )

        unet.set_attn_processor(lora_attn_procs)
        return lora_attn_procs

    def train(self, train_data_dir, num_epochs=100, learning_rate=1e-4):
        """
        Training logic using Diffusers trainer
        This is simplified - use diffusers' DreamBooth training script for production
        """
        from accelerate import Accelerator

        # Load base model
        pipe = StableDiffusionPipeline.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16
        )

        # Setup LoRA
        unet = pipe.unet
        lora_layers = self.setup_lora_layers(unet, rank=4)

        # Training loop would go here
        # For POC, use diffusers' train_dreambooth_lora.py script

        print(f"Training on {train_data_dir} for {num_epochs} epochs...")
        print("Use: accelerate launch train_dreambooth_lora.py \\")
        print(f"  --pretrained_model_name_or_path={self.model_id} \\")
        print(f"  --instance_data_dir={train_data_dir} \\")
        print(f"  --output_dir={self.output_dir} \\")
        print(f"  --instance_prompt='a photo of {self.concept_name}' \\")
        print(f"  --resolution=512 \\")
        print(f"  --train_batch_size=1 \\")
        print(f"  --gradient_accumulation_steps=4 \\")
        print(f"  --learning_rate={learning_rate} \\")
        print(f"  --lr_scheduler='constant' \\")
        print(f"  --max_train_steps={num_epochs * 10} \\")
        print(f"  --use_8bit_adam")
